## Reading raster data

This example shows verrious commonly used options to read single or multiple file raster datasets into an `xarray.Dataset` or `xarray.DataArray` object with geospatial attributes.

In **hydroMT** we typically we read data using the `DataCatalog` which allows for some minimal pre-processing in order to get uniform variable names and units. Here we show the methods that are used *under the hood* by the `DataCatalog.getrasterdataset` method. 

In [1]:
import numpy as np
import xarray as xr
from pprint import pprint
import glob
import os
import hydromt

In [2]:
# setup logging
from hydromt.log import setuplog

logger = setuplog("read raster data", log_level=10)

2021-10-13 11:25:42,180 - read raster data - log - INFO - HydroMT version: 0.4.4.dev


In [3]:
# Download artifacts for the Piave basin to `~/.hydromt_data/`.
data_catalog = hydromt.DataCatalog(logger=logger)
data_catalog.from_artifacts()

2021-10-13 11:25:42,188 - read raster data - data_adapter - INFO - Adding sample data v0.0.5 from artifacts


As an example we will use the [MERIT Hydro](http://hydro.iis.u-tokyo.ac.jp/~yamadai/MERIT_Hydro) dataset from the downloaded artifacts. This data is saved using various GeoTIFF files with identical grids in one folder.

In [4]:
path = os.path.join(os.path.dirname(data_catalog["merit_hydro"].path), "*.tif")
fns = glob.glob(path)
fns

['/home/runner/.hydromt_data/data/v0.0.5/merit_hydro/lndslp.tif',
 '/home/runner/.hydromt_data/data/v0.0.5/merit_hydro/flwdir.tif',
 '/home/runner/.hydromt_data/data/v0.0.5/merit_hydro/rivwth.tif',
 '/home/runner/.hydromt_data/data/v0.0.5/merit_hydro/elevtn.tif',
 '/home/runner/.hydromt_data/data/v0.0.5/merit_hydro/strord.tif',
 '/home/runner/.hydromt_data/data/v0.0.5/merit_hydro/upgrid.tif',
 '/home/runner/.hydromt_data/data/v0.0.5/merit_hydro/basins.tif',
 '/home/runner/.hydromt_data/data/v0.0.5/merit_hydro/hnd.tif',
 '/home/runner/.hydromt_data/data/v0.0.5/merit_hydro/uparea.tif']

### open_raster

To read raster data and parse it into an `xarray.DataArray` we use the `hydromt.open_raster` method. 
This method is based on `xarray.open_rasterio`, but additionally parses the coordinate reference system meta data. This method reads files from a single [gdal raster file](https://gdal.org/drivers/raster/index.html). Tiled data of a sinlge variable can also be passed as a [virtual raster tileset (vrt) file](https://gdal.org/drivers/raster/vrt.html).

In [5]:
# read a single raster file as DataArray
# the chunks argument provides lazy loading of the data, see xarray.open_rasterio
da = hydromt.open_raster(fns[0], chunks={"x": 1000, "y": 1000})
print(da)

<xarray.DataArray (y: 1920, x: 1680)>
dask.array<getitem, shape=(1920, 1680), dtype=float32, chunksize=(1000, 1000), chunktype=numpy.ndarray>
Coordinates:
  * y            (y) float64 46.8 46.8 46.8 46.8 46.8 ... 45.2 45.2 45.2 45.2
  * x            (x) float64 11.6 11.6 11.6 11.6 11.6 ... 13.0 13.0 13.0 13.0
    spatial_ref  int64 1
Attributes:
    AREA_OR_POINT:  Area
    _FillValue:     -9999.0


#### (geospatial) attributes

Many (geospatial) attributes can be accessed trough the DataArray/Dataset [raster accessors](https://deltares.github.io/hydromt/latest/api/api_methods.html#attributes)

In [6]:
# coordinate reference system
da.raster.crs

CRS.from_epsg(4326)

In [7]:
# geospatial transform, see https://www.perrygeo.com/python-affine-transforms.html
da.raster.transform

Affine(0.0008333333333333198, 0.0, 11.600000000000023,
       0.0, -0.000833333333333334, 46.8)

In [8]:
# names of x- and y dimensions
(da.raster.x_dim, da.raster.y_dim)

('x', 'y')

In [9]:
# nodata value (or fillvalue)
da.raster.nodata

-9999.0

### open_mfraster

To read multiple raster files with identical grid, but each with a different variable, into a single `xarray.Dataset` we can use the `hydromt.open_mfraster` method. The same method can be used to concatenate multiple raster files with identical grid and the same variable but different *layer* along a single dimension.

In [10]:
# this method takes both a list of paths or a path with a glob.glob pattern such as used here:
print(path)
ds = hydromt.open_mfraster(path, chunks={"x": 1000, "y": 1000})
ds

/home/runner/.hydromt_data/data/v0.0.5/merit_hydro/*.tif


<xarray.Dataset>
Dimensions:      (y: 1920, x: 1680)
Coordinates:
  * y            (y) float64 46.8 46.8 46.8 46.8 46.8 ... 45.2 45.2 45.2 45.2
  * x            (x) float64 11.6 11.6 11.6 11.6 11.6 ... 13.0 13.0 13.0 13.0
    spatial_ref  int64 1
Data variables:
    lndslp       (y, x) float32 dask.array<chunksize=(1000, 1000), meta=np.ndarray>
    flwdir       (y, x) uint8 dask.array<chunksize=(1000, 1000), meta=np.ndarray>
    rivwth       (y, x) float32 dask.array<chunksize=(1000, 1000), meta=np.ndarray>
    elevtn       (y, x) float32 dask.array<chunksize=(1000, 1000), meta=np.ndarray>
    strord       (y, x) uint8 dask.array<chunksize=(1000, 1000), meta=np.ndarray>
    upgrid       (y, x) int32 dask.array<chunksize=(1000, 1000), meta=np.ndarray>
    basins       (y, x) uint32 dask.array<chunksize=(1000, 1000), meta=np.ndarray>
    hnd          (y, x) float32 dask.array<chunksize=(1000, 1000), meta=np.ndarray>
    uparea       (y, x) float32 dask.array<chunksize=(1000, 1000), meta=np.ndarray>
Attributes:
    AREA_OR_POINT:  Area
    _FillValue:     -9999.0
    source_file:    lndslp.tif

TIP: To write a dataset back to a stack of raster in a single folder use the `<dataset>.raster.to_mapstack` method.

To concatenate multiple layers of [soilgrids data](https://www.isric.org/explore/soilgrids/faq-soilgrids-2017) into a single-variable dataset using `hydromt.open_mfraster`, we simply need to set the argument `concat=True` and optionally providing a `condat_dim` dimension name:

In [11]:
path = os.path.join(os.path.dirname(data_catalog["soilgrids"].path), "bd*.tif")
fns = glob.glob(path)
fns

['/home/runner/.hydromt_data/data/v0.0.5/soilgrids/bd_sl5.tif',
 '/home/runner/.hydromt_data/data/v0.0.5/soilgrids/bd_sl2.tif',
 '/home/runner/.hydromt_data/data/v0.0.5/soilgrids/bd_sl1.tif',
 '/home/runner/.hydromt_data/data/v0.0.5/soilgrids/bd_sl6.tif',
 '/home/runner/.hydromt_data/data/v0.0.5/soilgrids/bd_sl4.tif',
 '/home/runner/.hydromt_data/data/v0.0.5/soilgrids/bd_sl7.tif',
 '/home/runner/.hydromt_data/data/v0.0.5/soilgrids/bd_sl3.tif']

In [12]:
ds = hydromt.open_mfraster(fns, concat=True, concat_dim="layer")
ds

<xarray.Dataset>
Dimensions:      (y: 768, x: 672, layer: 7)
Coordinates:
  * y            (y) float64 46.8 46.8 46.79 46.79 ... 45.21 45.2 45.2 45.2
  * x            (x) float64 11.6 11.6 11.61 11.61 ... 12.99 12.99 13.0 13.0
    spatial_ref  int64 1
  * layer        (layer) int64 0 1 2 3 4 5 6
Data variables:
    bd           (layer, y, x) int16 dask.array<chunksize=(1, 768, 672), meta=np.ndarray>

### open_raster_from_tindex

If the raster data is tiled but for each tile a different CRS is used (for instance a different UTM projection for each UTM zone), this dataset cannot be described using a VRT file. In this case a vector file can be build to be used a raster tile index using [gdaltindex](https://gdal.org/programs/gdaltindex.html) and read using `hydromt.open_raster_from_tindex`. To read the data into a single `xarray.Dataset` the data needs to be reprojected and mosaiced to a single CRS while reading. As this type of data cannot be loaded lazily the method is typically used with an area of interest for which the data is loaded and combined. As example we use the [GRWL mask](https://doi.org/10.5281/zenodo.1297434) raster tiles for which we have created a tileindex using the aforementiond `gdaltindex` command line tool.

In [13]:
# area of interest based previously loaded soilgrids data bounding box
bbox = ds.raster.bounds
print(bbox)

(11.599969344082936, 45.19917341398322, 12.999969120083549, 46.79917315798391)


In [14]:
# the tileindex is a GeoPackage vector file
# with an attribute column 'location' containing the relative paths to the raster file data
import geopandas as gpd

fn_tindex = data_catalog["grwl_mask"].path
print(fn_tindex)
gpd.read_file(fn_tindex, rows=5)

/home/runner/.hydromt_data/data/v0.0.5/grwl_tindex.gpkg


,location,EPSG,geometry
0,GRWL_mask_V01.01//NA17.tif,EPSG:32617,"POLYGON ((-80.92667 4.00635, -77.99333 4.00082..."
1,GRWL_mask_V01.01//NA18.tif,EPSG:32618,"POLYGON ((-78.00811 4.00082, -71.99333 4.00082..."
2,GRWL_mask_V01.01//NA19.tif,EPSG:32619,"POLYGON ((-72.00811 4.00082, -65.99333 4.00082..."
3,GRWL_mask_V01.01//NA20.tif,EPSG:32620,"POLYGON ((-66.00811 4.00082, -59.99333 4.00082..."
4,GRWL_mask_V01.01//NA21.tif,EPSG:32621,"POLYGON ((-60.00811 4.00082, -53.99333 4.00082..."


In [15]:
# set destination CRS to EPSG:32633 (UTM zone 33N) to keep a projected crs
ds = hydromt.open_raster_from_tindex(
    fn_tindex, bbox=bbox, nodata=0, mosaic_kwargs={"dst_crs": 32633}
)
ds

<xarray.Dataset>
Dimensions:      (y: 6048, x: 3814)
Coordinates:
  * y            (y) float64 5.188e+06 5.188e+06 ... 5.007e+06 5.007e+06
  * x            (x) float64 2.33e+05 2.33e+05 2.33e+05 ... 3.473e+05 3.474e+05
    spatial_ref  int64 1
    mask         (y, x) bool False False False False ... False False False False
Data variables:
    NL33         (y, x) uint8 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0